In [2]:
!pip install keras-tuner

     |████████████████████████████████| 71kB 8.7MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.2-cp37-none-any.whl size=78938 sha256=a101c53418477018b4afefb9694ca22c14e43e82315323d76de135692bac3758
  Stored in directory: /root/.cache/pip/wheels/bb/a1/8a/7c3de0efb3707a1701b36ebbfdbc4e67aedf6d4943a1f463d6
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp37-none-any.whl size=15356 sha256=4882cf8265a7a6784ad5303561827487c129b860b82e87d9b39584976634e143
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


In [3]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [4]:
print(tf.__version__)

2.4.1


In [5]:
fashion_mnist=keras.datasets.fashion_mnist

In [6]:
(train_images,train_lables),(test_images,test_labels)=fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [7]:
#scaling data (they are already grayscale , scale in 0 to 1)

train_images =train_images/255.0
test_images =test_images/255.0

In [8]:
#reshape data initial size is 28*28

train_images[0].shape

(28, 28)

In [9]:
train_images =train_images.reshape(len(train_images),28,28,1)
test_images =test_images.reshape(len(test_images),28,28,1)

In [10]:
def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model

In [11]:
#Randomsearch over parameters
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters


In [12]:
tuner_search= RandomSearch(build_model,objective='val_accuracy',max_trials=5,directory='output',project_name ='fashion_mnest')

In [13]:
tuner_search.search(train_images,train_lables,epochs=3,validation_split=0.1) #bydefault 2 epochs

Trial 5 Complete [00h 00m 20s]
val_accuracy: 0.8636666536331177

Best val_accuracy So Far: 0.9048333168029785
Total elapsed time: 00h 02m 21s
INFO:tensorflow:Oracle triggered exit


In [14]:
model =tuner_search.get_best_models(num_models=1)[0]

In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 24, 24, 96)        2496      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 20, 20, 48)        115248    
_________________________________________________________________
flatten (Flatten)            (None, 19200)             0         
_________________________________________________________________
dense (Dense)                (None, 64)                1228864   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                650       
Total params: 1,347,258
Trainable params: 1,347,258
Non-trainable params: 0
_________________________________________________________________


In [17]:
model.fit(train_images,train_lables,epochs=10,validation_split=0.1,initial_epoch=4)

Epoch 5/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0524 - accuracy: 0.9806 - val_loss: 0.4601 - val_accuracy: 0.9097
Epoch 6/10
1688/1688 [==============================] - 8s 4ms/step - loss: 0.0433 - accuracy: 0.9843 - val_loss: 0.5060 - val_accuracy: 0.9088
Epoch 7/10
1688/1688 [==============================] - 8s 4ms/step - loss: 0.0367 - accuracy: 0.9865 - val_loss: 0.5204 - val_accuracy: 0.9068
Epoch 8/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0339 - accuracy: 0.9876 - val_loss: 0.5014 - val_accuracy: 0.9097
Epoch 9/10
1688/1688 [==============================] - 8s 4ms/step - loss: 0.0267 - accuracy: 0.9906 - val_loss: 0.5386 - val_accuracy: 0.9085
Epoch 10/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0279 - accuracy: 0.9904 - val_loss: 0.6451 - val_accuracy: 0.9075
